In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# ChromeDriver 경로 설정
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# 시작 URL
url = 'https://m.nhqv.com/customer/dicWord/list?sBoardSno=2#'
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기

# 전체 페이지의 용어와 설명을 저장할 리스트
all_terms = []
# 이미 크롤링한 용어를 저장할 집합
crawled_terms = set()

# 용어 크롤링 시작
while True:
    # 현재 페이지에서 모든 용어 항목을 가져오기
    terms = driver.find_elements(By.CSS_SELECTOR, 'ul#dicWordList > li > a.click_area > span.tbox2')
    
    # 현재 페이지에서 크롤링한 용어 리스트
    current_page_terms = []

    # 각 용어 클릭하여 설명 가져오기
    for term in terms:
        # 현재 용어 텍스트
        term_text = term.text.strip()
        
        # 이미 크롤링한 용어인지 확인
        if term_text in crawled_terms:
            continue  # 중복된 용어는 건너뜀

        print(f"용어: {term_text}")

        # JavaScript를 사용하여 용어 클릭
        driver.execute_script("arguments[0].click();", term)
        time.sleep(2)  # 설명 로드 대기

        # 연관어 가져오기
        related_terms = []
        try:
            related_elements = driver.find_elements(By.CSS_SELECTOR, 'div[style="padding-left:52px"] > a.c_bl1.f13.lineH18.tdeco')
            related_terms = [elem.text for elem in related_elements]
            print(f"연관어: {related_terms}\n")
        except NoSuchElementException:
            print("연관어를 찾을 수 없습니다.\n")

        # 설명 가져오기
        try:
            # 설명이 있는 div의 텍스트를 추출
            description = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'div#sVblDsc'))
            ).text
            print(f"설명: {description}\n")
        except (NoSuchElementException, TimeoutException):
            description = "설명 없음"
            print("설명을 찾을 수 없습니다.\n")

        # 용어, 설명, 연관어 저장
        all_terms.append({'용어': term_text, '연관어': ", ".join(related_terms), '설명': description})
        # 크롤링한 용어 추가
        crawled_terms.add(term_text)
        current_page_terms.append(term_text)  # 현재 페이지 용어 리스트에 추가

        # 목록으로 돌아가기 버튼 클릭
        try:
            back_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//p[span[text()='목록']]"))
            )
            driver.execute_script("arguments[0].click();", back_button)
            time.sleep(2)  # 목록 페이지 로딩 대기
        except (NoSuchElementException, TimeoutException, ElementNotInteractableException):
            print("목록으로 돌아가기 버튼을 찾을 수 없습니다.")
            break

    # 다음 페이지로 이동 (더보기 버튼 클릭)
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//p[span[text()='더보기']]"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # 새로운 페이지 로딩 대기
    except (NoSuchElementException, TimeoutException, ElementNotInteractableException):
        print("더보기 버튼을 찾을 수 없습니다. 모든 페이지를 다 크롤링했습니다.")
        break

# 크롤링 종료 후 브라우저 닫기
driver.quit()

# 크롤링한 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_terms)

# 데이터프레임을 엑셀로 저장
output_file = 'crawled_terms.xlsx'  # 저장할 파일명
df.to_excel(output_file, index=False, engine='openpyxl')
print(f"크롤링한 데이터가 '{output_file}'로 저장되었습니다.")


용어: 예수금
연관어: ['출금가능금액']

설명: 현재 오늘 날짜(D일) 기준으로 계좌에 있는 수표를 포함한 현금잔액을 의미합니다. 내일 날짜 현금잔액은 D+1 예수금, 모레 날짜 현금잔액은 D+2일 예수금이라 합니다. 만일 주식 매매 체결 후 돈이 필요해서 얼마만큼 출금할 수 있는지는 D+2 예수금을 확인하시면 됩니다.

증권거래는 매매 체결일(불포함) 이후 2번째 영업일에 결제됩니다(매수한 경우 출금, 매도한 경우 입금). 쉽게 설명하자면 후불결제 개념이죠. 이런 결제 방식으로 인해, 거래가 매일 일어나게 되면 예수금은 매일 달라져요. 그리고 주식 매매 결제는 휴일을 제외한 영업일 기준으로 계산됩니다.

증권계좌에 100만원을 입금하면 당일 예수금은 100만원이 됩니다. 만일 같은날 주식을 30만원어치 산다면 계좌의 예수금은 어떻게 될까요?
→D+1예수금은 아직 결제가 일어나기 전이기 때문에 100만원으로, D+2예수금은 결제예정일이기 때문에 70만원으로 표시됩니다.

용어: CMA RP
연관어: ['CMA 발행어음', 'CMA MMW', 'CMA MMF']

설명: 하루만 넣어도 이자를 받을 수 있고 원할 때 수시로 편리하게 입출금이 가능한 종합 자산관리계좌입니다. CMA 계좌에서는 은행 보통 예금 통장처럼 급여이체, 신용카드 및 공과금 납부 등도 가능합니다. 그리고 주식, 채권, 펀드 등 금융상품 거래가 가능하죠. 계좌개설 시 투자자는 이자를 받기 위해 운용되는 상품을 지정하게 됩니다. CMA RP로 선택하시고 입금하시면, RP로 자동적으로 투자가 되며 이익금은 투자자에게 지급됩니다.

RP란 증권사가 보유하고 있는 채권을 일정한 기간 후에 일정한 가격으로 다시 매수할 것을 약속한 채권입니다. 보통 1달~1년 이하의 단기 채권이기 때문에 현금화가 용이한 상품이죠.

투자자는 이자를 받기 위해 운용되는 상품을(=자동운용상품) RP가 아닌 다른 상품으로 변경할 수 있어요. MTS 에서 'CMA 약정'을 검색하시면, 보다 세부내용을 확인하실 수 있어요.



In [3]:
# 데이터프레임을 엑셀로 저장
output_file = 'crawled_terms.xlsx'  # 저장할 파일명
df.to_excel(output_file, index=False, engine='openpyxl')
print(f"크롤링한 데이터가 '{output_file}'로 저장되었습니다.")

크롤링한 데이터가 'crawled_terms.xlsx'로 저장되었습니다.
